In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"
import findspark
findspark.init()
from google.colab import files
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import isnan, when, count, col, lit
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder

In [3]:
sc = SparkSession.builder.master('local[*]').getOrCreate()

In [4]:
!ls

drive  sample_data  spark-2.4.7-bin-hadoop2.7  spark-2.4.7-bin-hadoop2.7.tgz


In [5]:
data = sc.read.csv('drive/My Drive/Colab Notebooks/dataset/pyspark_data.csv', inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Engine Fuel Type: string (nullable = true)
 |-- Engine HP: integer (nullable = true)
 |-- Engine Cylinders: integer (nullable = true)
 |-- Transmission Type: string (nullable = true)
 |-- Driven_Wheels: string (nullable = true)
 |-- Number of Doors: integer (nullable = true)
 |-- Market Category: string (nullable = true)
 |-- Vehicle Size: string (nullable = true)
 |-- Vehicle Style: string (nullable = true)
 |-- highway MPG: integer (nullable = true)
 |-- city mpg: integer (nullable = true)
 |-- Popularity: integer (nullable = true)
 |-- MSRP: integer (nullable = true)



In [7]:
data.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
Make,11914,None,None,Acura,Volvo
Model,11914,745.5822222222222,1490.8280590623795,1 Series,xD
Year,11914,2010.384337753903,7.5797398875957995,1990,2017
Engine Fuel Type,11911,None,None,diesel,regular unleaded
Engine HP,11845,249.38607007176023,109.19187025917194,55,1001
Engine Cylinders,11884,5.628828677213059,1.78055934824622,0,16
Transmission Type,11914,None,None,AUTOMATED_MANUAL,UNKNOWN
Driven_Wheels,11914,None,None,all wheel drive,rear wheel drive
Number of Doors,11908,3.4360933825999327,0.8813153865835529,2,4


In [9]:
def replace(column, value):
  return when(column != value, column).otherwise((lit(None)))


In [10]:
data = data.withColumn('Market Category', replace(col('Market Category'), 'N/A'))

In [13]:
data.select([count(when(isnan(c)| col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+
|Make|Model|Year|Engine Fuel Type|Engine HP|Engine Cylinders|Transmission Type|Driven_Wheels|Number of Doors|Market Category|Vehicle Size|Vehicle Style|highway MPG|city mpg|Popularity|MSRP|
+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+
|   0|    0|   0|               3|       69|              30|                0|            0|              6|           3742|           0|            0|          0|       0|         0|   0|
+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+



In [15]:
data.rdd.getNumPartitions()

1

In [16]:
data = data.drop('Market Category')
data = data.na.drop()
print((data.count(), len(data.columns)))

(11812, 15)


In [17]:
assembler = VectorAssembler(inputCols=['Year', 'Engine HP', 'Engine Cylinders', 'Number of Doors', 'highway MPG', 'city mpg', 'Popularity'],
                            outputCol='Attributes')


In [19]:
regressor = RandomForestRegressor(featuresCol='Attributes', labelCol='MSRP')

In [21]:
pipeline = Pipeline(stages=[assembler, regressor])
pipeline.write().overwrite().save('pipeline')


In [22]:
pipelineModel = Pipeline.load('pipeline')


In [24]:
paramGrid = ParamGridBuilder().addGrid(regressor.numTrees, [100, 500]).build()

In [25]:
crossval = CrossValidator(estimator=pipelineModel, estimatorParamMaps=paramGrid, evaluator=RegressionEvaluator(labelCol='MSRP'), numFolds=3)


In [26]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=123)
cvModel = crossval.fit(train_data)

In [27]:
bestModel = cvModel.bestModel
for i in range(len(bestModel.stages)):
  print(bestModel.stages[i])

VectorAssembler_4cfbfbb20a01
RandomForestRegressionModel (uid=RandomForestRegressor_0ab88255c8b8) with 500 trees


In [28]:
pred = cvModel.transform(test_data)

In [30]:
pred.select('MSRP', 'prediction').show()

+-----+------------------+
| MSRP|        prediction|
+-----+------------------+
|29980|32663.872687140803|
|28030|32480.622110577813|
|30030|32480.622110577813|
|32700|36800.677804647086|
|29350|23494.443639116023|
|31890| 27561.99412744512|
|34980| 27561.99412744512|
| 2799| 5068.925188068261|
| 2827| 5116.216138280437|
| 3381| 5943.321887550303|
|24450| 26705.70647257413|
|21050| 22907.87930596402|
| 2000| 5274.076931229702|
| 2181| 7896.335147218928|
| 2144| 5240.909783710902|
| 2265| 7904.501978811256|
|56780|39378.819909292135|
|49440| 39340.95848398953|
|50640| 39340.95848398953|
|52640| 39340.95848398953|
+-----+------------------+
only showing top 20 rows



In [31]:
eval = RegressionEvaluator(labelCol='MSRP')
rmse = eval.evaluate(pred)
mse = eval.evaluate(pred, {eval.metricName: 'mse'})

In [32]:
rmse

16532.27243376136

In [33]:
mse

273316031.8241058